In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
# Using the crop types of the previous seven years to predict the crop types for 2023
year_to_predict = 2023
"""end_year = year_to_predict-1
start_year = end_year-7

columns_to_check = [f'crop_{year}' for year in range(start_year + 1, end_year + 1)]
print(columns_to_check)
"""
columns_to_check = ['2023']

In [3]:
# Folder path and file path
CDL_folder_path = "/data/Anonymous/Anonymousfile/best_practice/data/trusted_pixels_in_US_and_LUCAS"
CDL_file_path = site_id+'_cdl_samples_bound_buffered.csv'
# CDL_file_path = '28_29_cdl_samples_bound_buffered.csv'

# Join the folder path and file path using os.path.join
full_path = os.path.join(CDL_folder_path, CDL_file_path)

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(full_path)
df.head()


,system:index,cropland,latitude,longitude,.geo
0,2013_0,1,44.550960,-95.411784,"{""type"":""Point"",""coordinates"":[-95.41178417798..."
1,2013_1,195,44.839965,-96.103567,"{""type"":""Point"",""coordinates"":[-96.10356680888..."
2,2013_2,1,45.188304,-95.505555,"{""type"":""Point"",""coordinates"":[-95.50555530433..."
3,2013_3,5,44.780556,-95.983092,"{""type"":""Point"",""coordinates"":[-95.98309201540..."
4,2013_4,5,44.442522,-94.244866,"{""type"":""Point"",""coordinates"":[-94.24486608978..."


In [4]:
df['year'] = df['system:index'].str.slice(0, 4)
df['sample_id'] = df['system:index'].str.slice(5,)
df = df.loc[:, ['sample_id', 'year','latitude','longitude','cropland']]
df.head()

,sample_id,year,latitude,longitude,cropland
0,0,2013,44.550960,-95.411784,1
1,1,2013,44.839965,-96.103567,195
2,2,2013,45.188304,-95.505555,1
3,3,2013,44.780556,-95.983092,5
4,4,2013,44.442522,-94.244866,5


In [5]:
df_2023 = df[df['year'] == '2023']
df_2023.head()

,sample_id,year,latitude,longitude,cropland
10000000,0,2023,44.550960,-95.411784,1
10000001,1,2023,44.839965,-96.103567,195
10000002,2,2023,45.188304,-95.505555,1
10000003,3,2023,44.780556,-95.983092,5
10000004,4,2023,44.442522,-94.244866,5


In [6]:
# Group by the 'sample_id' column
grouped = df.groupby('sample_id')

# Aggregate multiple functions
agg_grouped = grouped.agg({
    'latitude': 'first', # Assuming 'latitude' is the same for the same 'sample_id'
    'longitude': 'first' # Assuming 'longitude' is the same for the same 'sample_id'
})

# Use apply to combine all cropland values into a list
agg_grouped['cropland_all'] = grouped['cropland'].apply(list)
agg_grouped['year_all'] = grouped['year'].apply(list)
agg_grouped.reset_index(inplace=True)

In [7]:

crop_2023 = [agg_grouped.iloc[i]['cropland_all'][10] for i in range(len(agg_grouped))]

new_df = agg_grouped[['sample_id','latitude','longitude']].copy()

new_df['crop_2023'] = crop_2023
del agg_grouped

In [8]:
new_df.head()

,sample_id,latitude,longitude,crop_2023
0,0,44.550960,-95.411784,1
1,1,44.839965,-96.103567,195
2,10,44.564679,-95.371450,1
3,100,45.477924,-95.783639,1
4,1000,44.110942,-94.226726,131


In [9]:
corn_1 = new_df[new_df['crop_2023'] == 1]
soybean_5 = new_df[new_df['crop_2023'] == 5]
other_10 = new_df[(new_df['crop_2023']!=1) & (new_df['crop_2023']!=5)]
corn_1.shape,soybean_5.shape,other_10.shape

((394785, 4), (300595, 4), (304620, 4))

In [10]:
corn_1['label']=1
soybean_5['label']=5
other_10['label']=10

/home/jlong2/anaconda3/envs/gee37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jlong2/anaconda3/envs/gee37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jlong2/anaconda3/envs/gee37/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [11]:
# Output corn and soybean trusted pixels
# corn_1[['latitude','longitude','label']].to_csv(os.path.join(CDL_folder_path, site_id+'_'+str(year_to_predict)+'_corn_2023_cdl.csv'),index=False)
# soybean_5[['latitude','longitude','label']].to_csv(os.path.join(CDL_folder_path, site_id+'_'+str(year_to_predict)+'_soybean_2023_cdl.csv'),index=False)
# other_10[['latitude','longitude','label']].to_csv(os.path.join(CDL_folder_path, site_id+'_'+str(year_to_predict)+'_other_2023_cdl.csv'),index=False)

In [12]:
frames = [corn_1, soybean_5, other_10]

cdl_samples = pd.concat(frames)
cdl_samples[['latitude','longitude','label']].to_csv(os.path.join(CDL_folder_path, site_id+'_'+str(year_to_predict)+'_'+str(year_to_predict)+'_cdl_samples.csv'),index=False)
